In [1]:
import numpy as np
import pandas as pd

from PIL import Image 
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import os
import re
import cv2

import skimage
from sklearn.base import BaseEstimator, TransformerMixin
from skimage.feature import hog

from scipy.sparse import csr_matrix
import h5py


In [2]:
class GrayScaleConverter(BaseEstimator, TransformerMixin):
    """
    Convert an array of RGB images to grayscale
    """
 
    def __init__(self):
        pass
 
    def fit(self, X, y=None):
        """returns itself"""
        return self
 
    def transform(self, X, y=None):
        """perform the transformation and return an array"""
        return np.array([skimage.color.rgb2gray(img) for img in X])
        

In [3]:
class HogTransformer(BaseEstimator, TransformerMixin):
    """
    Expects an array of 2d arrays (1 channel images)
    Calculates hog features for each img
    """
 
    def __init__(self, y=None, orientations=9,
                 pixels_per_cell=(14,14),
                 cells_per_block=(2,2), block_norm='L2-Hys'):
        self.y = y
        self.orientations = orientations
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block
        self.block_norm = block_norm
 
    def fit(self, X, y=None):
        return self
 
    def transform(self, X, y=None):
 
        def local_hog(X):
            return hog(X,
                       orientations=self.orientations,
                       pixels_per_cell=self.pixels_per_cell,
                       cells_per_block=self.cells_per_block,
                       block_norm=self.block_norm)
 
        try: # parallel
            return np.array([local_hog(img) for img in X])
        except:
            return np.array([local_hog(img) for img in X])
            

In [4]:
class CannyEdgeDetector(BaseEstimator, TransformerMixin):
    
    def __init__(self, gaussian_kernel_size=(3, 3), gaussian_sigma=0, canny_low_threshold=100, canny_high_threshold=200):
        self.gaussian_kernel_size = gaussian_kernel_size
        self.gaussian_sigma = gaussian_sigma
        self.canny_low_threshold = canny_low_threshold
        self.canny_high_threshold = canny_high_threshold
    
    def fit(self, X, y=None):
        return self
 
    def transform(self, X, y=None):
        edged_images = []
        for image in X:
            # Appliquer le filtre gaussien
            blurred_image = cv2.GaussianBlur(np.uint8(image), self.gaussian_kernel_size, self.gaussian_sigma)
            
            # Appliquer le filtre Canny
            edges = cv2.Canny(blurred_image, self.canny_low_threshold, self.canny_high_threshold)
            
            # Ajouter les bords détectés à l'image d'origine
            edged_image = cv2.bitwise_and(image, image, mask=edges)
            
            edged_images.append(edged_image)
            
        return np.array(edged_images)



In [5]:
class LaplacianFilter(BaseEstimator, TransformerMixin):
    def __init__(self, ddepth=cv2.CV_64F):
        self.ddepth = ddepth
    
    def fit(self, X, y=None):
        return self
 
    def transform(self, X, y=None):
        laplacian_images = []
        for image in X:
            # Appliquer le filtre Laplacien
            laplacian_image = cv2.Laplacian(np.uint8(image), self.ddepth)
            
            # Convertir l'image en 3 canaux si nécessaire
            if len(image.shape) == 3:
                laplacian_image = cv2.cvtColor(laplacian_image, cv2.COLOR_GRAY2BGR)
            
            laplacian_images.append(laplacian_image)
            
        return np.array(laplacian_images)


In [6]:
class HSVHistogram(BaseEstimator, TransformerMixin):
    def __init__(self, bins=256):
        self.bins = bins
    
    def fit(self, X, y=None):
        return self
 
    def transform(self, X, y=None):
        histograms = []
        for image in X:
            
            # Assurez-vous que l'image est en format flottant
            image = image.astype(np.float32)

            # Normaliser l'image pour obtenir des valeurs entre 0 et 1
            image /= 255.0

            # Diviser l'image en canaux de couleur
            channels = cv2.split(image)

            # Initialiser un histogramme vide pour chaque canal de couleur
            hist = []

            # Calculer l'histogramme pour chaque canal de couleur
            for channel in channels:
                hist_channel, _ = np.histogram(channel, bins=256, range=(0, 1))
                hist.append(hist_channel)

            # Concaténer les histogrammes de chaque canal pour obtenir l'histogramme global
            hist = np.concatenate(hist)
            histograms.append(hist)
            
        return np.array(histograms)


In [8]:
# Charger un tableau NumPy à partir du fichier HDF5
with h5py.File('data_0_comp_100.h5', 'r') as hf:
    X_images_train = hf['X_images_train'][:]
    

In [9]:
processor = GrayScaleConverter()
X_train_gray = processor.transform(X_images_train)

In [10]:
processor = HogTransformer()
X_train_hog = processor.transform(X_train_gray)

In [11]:
processor = CannyEdgeDetector()
X_train_canny = processor.transform(X_train_gray)

In [12]:
processor = LaplacianFilter()
X_train_laplacian = processor.transform(X_train_gray)

In [13]:
processor = HSVHistogram()
X_train_hsv = processor.transform(X_images_train)

In [14]:
# Ouvrir le fichier HDF5 existant en mode append
with h5py.File('data_4_comp.h5', 'r+') as hf:
    # Créer un nouvel ensemble de données dans le fichier HDF5
    hf.create_dataset('X_train_canny', data=X_train_canny, compression='gzip')
    hf.create_dataset('X_train_laplacian', data=X_train_laplacian, compression='gzip')
    hf.create_dataset('X_train_hog', data=X_train_hog, compression='gzip')
    hf.create_dataset('X_train_hsv', data=X_train_hsv, compression='gzip')
    

ValueError: Unable to create dataset (name already exists)

In [ ]:
del X_train
del X_train_canny
del X_train_laplacian
del X_train_hog
del X_train_hsv